**Desription**:
    
The following notebook is a revised version of the [Crowdvariant analysis notebook](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Machine%20Learning%5BAll%20Technologies%5D.ipynb). 

In the first part of this notebook, a random forest classifier was trained on crowdsourced[CrowdVariant](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Data%20Collection.ipynb) data for HG002 Deletions. 

In the second part of this notebook, the trained randfom forest classifier is then used to predict labels for a new set of randomly selected HG002 Deletions.

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



***
Load Data
***
Restructure features so that the training dataset and prediction dataset have all of the same features

In [2]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/Train/Final_DF/CrowdVar.Train_HG002.csv')
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/Train/Final_DF/CrowdVar.Train_HG002.csv')

In [ ]:
df_crowd.head(3)

In [7]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_data2.csv')
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/svviz.Annotate.DEL.HG002_data2.csv')

In [8]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd_2.columns.values)
d = list(HG002_pred.columns.values)
set(d) - set(c)

{'GIAB_Crowd',
 'Ill300x.amb_reason_insertSizeScore_orientation',
 'IllMP.amb_reason_insertSizeScore_alignmentScore',
 'SVtype',
 'Size',
 'TenX.HP1_amb_reason_insertSizeScore_insertSizeScore',
 'TenX.HP1_ref_reason_insertSizeScore',
 'TenX.HP2_amb_reason_insertSizeScore_insertSizeScore',
 'TenX.HP2_amb_reason_orientation_insertSizeScore',
 'id',
 'type'}

In [ ]:
# a = ['chrom','start','end','sample','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','size','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label','CN0_prob','CN1_prob','CN2_prob']
# b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

# set(b) - set(a)

In [9]:
### Training Set: Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)
# df_crowd.columns.drop(['Label'],axis=1, inplace = True)
df_crowd.drop(['size'],axis=1, inplace = True)
df_crowd.drop(['Ill250.amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)

In [ ]:
### Prediction Set: Drop irrelevant columns and GT information
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop('SVtype', axis=1)
HG002_pred.drop('type',axis=1)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
HG002_pred.drop(['Size'],axis=1, inplace = True)
HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
HG002_pred.drop(['Ill300x.amb_reason_insertSizeScore_orientation'],axis=1, inplace = True)
HG002_pred.drop(['IllMP.amb_reason_insertSizeScore_alignmentScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP1_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP1_ref_reason_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP2_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
HG002_pred.drop(['TenX.HP2_amb_reason_orientation_insertSizeScore'],axis=1, inplace = True)

In [ ]:
df_crowd.head(3)

*** 
Impute missing values using KNN 
***

In [10]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,25.500000,2.0,0.0,8,0.096911,0,0.0,0,0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,138.336019,12.0,0.0,2,0.221591,0,0.0,0,0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,181.532366,3.0,0.0,3,0.059443,0,0.0,0,0,0.0


In [11]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

Imputing row 1/1515 with 0 missing, elapsed time: 1.823
Imputing row 101/1515 with 0 missing, elapsed time: 1.873
Imputing row 201/1515 with 0 missing, elapsed time: 1.876
Imputing row 301/1515 with 0 missing, elapsed time: 1.881
Imputing row 401/1515 with 0 missing, elapsed time: 1.886
Imputing row 501/1515 with 0 missing, elapsed time: 1.888
Imputing row 601/1515 with 0 missing, elapsed time: 1.890
Imputing row 701/1515 with 0 missing, elapsed time: 1.891
Imputing row 801/1515 with 0 missing, elapsed time: 1.892
Imputing row 901/1515 with 0 missing, elapsed time: 1.893
Imputing row 1001/1515 with 0 missing, elapsed time: 1.893
Imputing row 1101/1515 with 0 missing, elapsed time: 1.898
Imputing row 1201/1515 with 0 missing, elapsed time: 1.901
Imputing row 1301/1515 with 0 missing, elapsed time: 1.901
Imputing row 1401/1515 with 0 missing, elapsed time: 1.901
Imputing row 1501/1515 with 0 missing, elapsed time: 1.902


In [12]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
dfcrowd_header = list(df_crowd.columns.values)

In [14]:
X.columns = dfcrowd_header
X.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,TenX.HP2_ref_insertSize_std,TenX.HP2_ref_reason_alignmentScore,TenX.HP2_ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,579.446154,17.934094,65.0,756.430769,163.879409,0.0,65.0,0.0,526.250785,88.797088,...,25.500000,2.0,0.0,8.0,0.096911,0.0,0.0,0.0,0.0,1.0
1,557.000000,20.665840,13.0,1158.307692,134.247982,0.0,13.0,0.0,545.050452,62.371767,...,138.336019,12.0,0.0,2.0,0.221591,0.0,0.0,0.0,0.0,1.0
2,574.335366,18.613946,164.0,678.518293,139.056203,31.0,133.0,0.0,518.296740,91.633718,...,181.532366,3.0,0.0,3.0,0.059443,0.0,0.0,0.0,0.0,0.0


In [15]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [16]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 1.0    959
 0.0    552
-1.0      1
 2.0      3
dtype: int64

In [17]:
# Remove labels from feature set
X.drop(['Label'],axis=1, inplace = True)

In [25]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

** Train RF Classifier **

In [26]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3, random_state=0)

In [27]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [28]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [29]:
model.predict(X_test)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,
        1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0

In [31]:
pred = model.predict(X_test)

In [32]:
precision_score(pred, y_test, average='micro') 

0.97802197802197799

In [33]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,pacbio.ref_count,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
1222,994.916667,5.415384,48.0,447.020833,92.768864,44.0,4.0,0.0,904.924419,131.254148,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.0,0.12500
310,970.742857,26.804416,35.0,460.485714,79.329465,34.0,1.0,0.0,881.921162,147.513942,...,34.0,10505.52941,3827.096344,34.0,0.0,0.0,0.0,0.000000,1.0,0.10625
9,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,886.094675,150.269472,...,25.0,9280.32000,4389.713227,25.0,0.0,0.0,0.0,0.000000,0.0,0.00000
785,978.500000,21.362350,20.0,425.550000,75.699719,19.0,1.0,0.0,903.137725,153.256269,...,30.0,11302.16667,5049.547201,30.0,0.0,0.0,1.0,0.404959,1.0,0.31405
295,955.318182,20.636213,22.0,446.727273,93.222696,21.0,1.0,0.0,894.035503,145.256675,...,26.0,10008.84615,3791.354088,26.0,0.0,0.0,0.0,0.000000,2.0,0.16358


In [35]:
labels = pd.DataFrame(y_test)

In [36]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [37]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0.0    166
1.0    288
2.0      1
dtype: int64

In [38]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0.0    165
1.0    290
dtype: int64

In [39]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[161   5   0]
 [  4 284   0]
 [  0   1   0]]


In [40]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0.0,161,5,166
1.0,4,284,288
2.0,0,1,1
All,165,290,455


In [51]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

             precision    recall  f1-score   support

        0.0       0.98      0.97      0.97       166
        1.0       0.98      0.99      0.98       288
        2.0       0.00      0.00      0.00         1

avg / total       0.98      0.98      0.98       455



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



***
Predict
***

**Load Data**

In [41]:
HG002_pred.head(3)

,Ill300x.alt_alnScore_mean,Ill300x.alt_alnScore_std,Ill300x.alt_count,Ill300x.alt_insertSize_mean,Ill300x.alt_insertSize_std,Ill300x.alt_reason_alignmentScore,Ill300x.alt_reason_insertSizeScore,Ill300x.alt_reason_orientation,Ill300x.amb_alnScore_mean,Ill300x.amb_alnScore_std,...,pacbio.ref_insertSize_mean,pacbio.ref_insertSize_std,pacbio.ref_reason_alignmentScore,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,541.033679,91.702755,...,9694.425532,4306.492796,47.0,0,0.0,0,0.000000,0,0,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,520.882022,78.521682,...,9218.592593,4009.865909,27.0,0,0.0,0,0.000000,0,0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,531.881724,77.118761,...,10296.150000,4547.097621,40.0,0,0.0,1,0.675712,0,0,2


In [42]:
X2 = HG002_pred

** Impute missing values using KNN **

In [43]:
# Convert dataframe to matrix
X2=X2.as_matrix()

#Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

Imputing row 1/2805 with 0 missing, elapsed time: 6.678
Imputing row 101/2805 with 0 missing, elapsed time: 6.684
Imputing row 201/2805 with 0 missing, elapsed time: 6.692
Imputing row 301/2805 with 0 missing, elapsed time: 6.702
Imputing row 401/2805 with 0 missing, elapsed time: 6.706
Imputing row 501/2805 with 0 missing, elapsed time: 6.707
Imputing row 601/2805 with 0 missing, elapsed time: 6.711
Imputing row 701/2805 with 0 missing, elapsed time: 6.714
Imputing row 801/2805 with 0 missing, elapsed time: 6.715
Imputing row 901/2805 with 0 missing, elapsed time: 6.716
Imputing row 1001/2805 with 0 missing, elapsed time: 6.721
Imputing row 1101/2805 with 0 missing, elapsed time: 6.724
Imputing row 1201/2805 with 0 missing, elapsed time: 6.724
Imputing row 1301/2805 with 0 missing, elapsed time: 6.725
Imputing row 1401/2805 with 0 missing, elapsed time: 6.727
Imputing row 1501/2805 with 0 missing, elapsed time: 6.730
Imputing row 1601/2805 with 0 missing, elapsed time: 6.740
Imputing 

In [44]:
X2.columns = list(HG002_pred.columns.values)

In [45]:
X2.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [46]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [47]:
model.predict(X5)

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

In [48]:
# Calculate Precision Score
true = HG002_pred['GIAB_Crowd']
predicted = model.predict(X5)
precision_score(true, predicted, average='micro') 

0.75258467023172904

In [49]:
pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,586,12,598
1,209,1525,1734
2,26,447,473
All,821,1984,2805


In [50]:
from sklearn.metrics import classification_report
true = HG002_pred['GIAB_Crowd']
pred = model.predict(X5)
print(classification_report(true, pred))

             precision    recall  f1-score   support

          0       0.71      0.98      0.83       598
          1       0.77      0.88      0.82      1734
          2       0.00      0.00      0.00       473

avg / total       0.63      0.75      0.68      2805



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



**TODO**

[Feature Importance](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)